# Generate trades using dedicated service - Python

### Overview
Retrieving trades over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request trades and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual trades among market activity. Please refer to <i>bestex-pointintime</i> and <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured trades information.

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matpotlib* as per display packages
    
***

# Run Trades sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 3, day = 5), 
    end_date = date.Date(year = 2021, month = 3, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 30, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [identifier],
    constraints = constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick trades service
        service = tick_trades_service.TickTradesServiceStub(channel)
        
        # process the tick trades request
        trades = []
        metadata = [('authorization', token)]
        for trade in service.TickTrades(request=request, metadata=metadata):
            if trade.trade:
                trades.append(trade.trade)

    # keep only trades, delete instrument mapping (1st element retrieved)
    # mapping is not used as only one instrument identifier is requested
    del trades[0]
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

### Step 4: Visualize data

#### 4.1 Retrieve data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in trades]
prices = [t.price for t in trades]
sizes = [t.size for t in trades]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d = {'Date': dates, 'Price': prices, 'Size': sizes}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot trades with matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2, label="Price")

# twin x-axis for two different y-axis
ax2=ax.twinx()
ax2.plot('Date', 'Size', data=df, marker='', color='red', linewidth=2, label="Size")

# set graph title and axis label
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="blue",fontsize=14)
ax2.set_ylabel("Size",color="red",fontsize=14)
plt.title('Tick trades for {0}-{1}'.format(ticker, exchange))
plt.show()

#### 4.3 Plot trades with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=False)
fig.add_trace(go.Bar(x=df['Date'], y=df['Size'], name="Size"), secondary_y=True)

# set graph title and axis label
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick trades for {0}-{1}'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()